In [1]:
import tensorflow as tf
import glob
import matplotlib.pyplot as plt
import numpy as np
import os

# --- CONFIGURATION ---
# Use the local path you confirmed works!
data_path = '/Users/abhicado/Library/CloudStorage/GoogleDrive-agoda.wonders@gmail.com/My Drive/CS6140_Project_Data/*.tfrecord.gz'
files = glob.glob(data_path)

feature_description = {
    'B1': tf.io.VarLenFeature(tf.float32),  # Blue
    'B2': tf.io.VarLenFeature(tf.float32),  # Blue
    'B3': tf.io.VarLenFeature(tf.float32),  # Blue
    'B4': tf.io.VarLenFeature(tf.float32),  # Blue
    'B5': tf.io.VarLenFeature(tf.float32),  # Blue
    'B6': tf.io.VarLenFeature(tf.float32),  # Blue
    'B7': tf.io.VarLenFeature(tf.float32),  # Blue
    'B8': tf.io.VarLenFeature(tf.float32),  # Blue
    'B8A': tf.io.VarLenFeature(tf.float32),  # Blue
    'B9': tf.io.VarLenFeature(tf.float32),  # Blue
    'B10': tf.io.VarLenFeature(tf.float32),  # Green
    'B11': tf.io.VarLenFeature(tf.float32),  # Red
    'B12': tf.io.VarLenFeature(tf.float32),  # Red
    'classification': tf.io.FixedLenFeature([], tf.float32),
}

def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

raw_dataset = tf.data.TFRecordDataset(files[400], compression_type='GZIP')
raw_dataset = raw_dataset.shuffle(raw_dataset.cardinality())
parsed_dataset = raw_dataset.map(_parse_function)

# Check shape of the first element
for parsed_record in parsed_dataset.take(1):
    # If image is raw bytes, you might need to decode it to see spatial dims
    image = tf.io.decode_image(parsed_record['image_raw'])
    print("Image Shape:", image.shape)
    print("Label Shape:", parsed_record['label'].shape)

2026-02-18 16:30:44.015446: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:390] TFRecordDataset `buffer_size` is unspecified, default to 262144


KeyError: 'image_raw'